# Chapter 5 - Exercise 3
### Author: *John Benedick Estrada*
---
**Exercise:** The goal of this exercise is to implement a Turing machine.

1. Read about Turing machines at http://en.wikipedia.org/wiki/Turing_machine.

2. Write a class called `Turing` that implements a Turing machine.  For the action table, use the rules for a 3-state busy beaver.

3. Write a `draw` method that plots the state of the tape and the position and state of the head.  For one example of what that might look like, see http://mathworld.wolfram.com/TuringMachine.html.

In [1]:
from Turing import Turing

## Turing machine
_NOTE: This was a hobby project I did in my spare time. While the parser seems to work on the programs I tested it with, there is no assurance that it will work on all programs. Possible edge cases have not yet been tested._

#### Description
`Turing` is an implementation of a universal Turing machine (TM). Its code implementation is in `Turing.py` and `TMParser.py`.

It can be initialized with at least one symbol. The tape can be arbitrarily extended. This variant of the Turing machine has the following instructions:
- Tape cell shift instructions
  - `shiftl` - Move to the left of the current tape cell.
  - `shiftr` - Move to the right of the current tape cell.
  - `noshift` - Stay in the current cell.
- Write instructions
  - `write SYMBOL` - Write the symbol `SYMBOL` onto the current cell.
  - `erase` - Erase the symbol on the current cell. Alias to the `write BLANK_SYM` where `BLANK_SYM` is the blank symbol.
- Change state instruction
  - `goto STATE` - Jump to the specified state `STATE`.

In this notebook, the Turing machine is implemented as the iterator class `Turing`.

------
#### Operation
The `Turing` class is initialized with the program for the Turing machine written in the language specified in the cell below. The program is represented as a string. For example
```
tm = Turing(tm_prog)
```
Before running the resulting instance (`tm` in our example), the initial contents of the tape must be loaded first. This is done by calling the `load_tape` method of the `Turing` class with the initial tape contents in the form of an iterable like `str` or `list`. The first element in the initial tape content is the first cell to be read. Passing `None` or passing no value to `load_tape` loads a blank tape to the Turing machine. In the following example, the `Turing` object `tm` is loaded with a non-blank tape in the form a `str` object.
```
tm.load_tape("000100")
```
To run the Turing machine, we iterate the `Turing` object with the `next` function or the with the `for` loop. Every iteration, two objects are returned: the current position of the TM on the tape, and the current state of the TM. These object can be used to determine the current state of the tape and the Turing machine.

The tape of a `Turing` object can be accessed with the the `tape` attribute. It is not protected (for instances with the `property` decorator), so users must be cautious in accessing `tape` so as not to accidentally modify its values.

In the following example, the Turing object `tm` is run with the for loop and its tape, and the current state printed.
```
for pos, state in tm:
    # Print the current state.
    print(f"State: {state}")
    # Print the tape centered around the current position of `tm` on the tape.
    print(tm.tape[i] for i in range(pos-10, pos+10))
```

`Turing` objects can be reset back to the initial state and the initial tape contents with the `reset` method.

------
#### Language for the Turing machine

A simple programming language was made for this implementation of the Turing machine. It is inspired by [this lecture note](https://web.stanford.edu/class/archive/cs/cs103/cs103.1132/lectures/19/Small19.pdf) I found on the internet. It details a language for a different but equivalent Turing machine called "Wang B-machine".

Code written in this language must have the following blocks:
```
[SYMBOL DECLARATION]
[STATE 1 DECLARATION]
[STATE 2 DECLARATION]
...
[STATE N DECLARATION]
```

White spaces are (should be) non-significant, that is you can use whatever indentation style you want, be it 4 spaces or tabs.

Comments start with the hash or pound symbol `#`.

The "symbol declaration" construct follows this syntax:
```
symbol:
    sym_1 (blank)
    sym_2
    sym_3
    ...
    sym_M
```

where `sym_i` for`i`$\in[1, M]$ are the symbols the Turing machine recognizes. Since cells on Turing machine tapes can be blank, one of the symbols declared must be the blank symbol. In this language, the first symbol to be declared must be the blank symbol. The blank symbol is declared with the string `(blank)`. Note that there must be at least one symbol declared.

The "state declaration" construct follows this syntax:
```
STATE_NAME:
    sym_1 -> instr_1; instr_2; ... instr_k1
    sym_2 -> instr_1; instr_2; ... instr_k2
    ...
    sym_M -> instr_1; instr_2; ... instr_kM
```
The string `STATE_NAME` is the name of the state with which it is identified by commands.

The line with the form `sym_i -> instr_1; instr_2; ... instr_ki` is a "rule". To the left of the arrow symbol `->` is a symbol, while to the right are the sequence of instructions the Turing machine will execute if the current symbol on the tape is the one on the left hand side of the "rule".

Simply put, "rules" `sym_i -> instr_1; instr_2; ... instr_ki` can be summarized with the following pseudo-code:
```
if the current symbol is `sym_i`:
    do `instr_1`
    do `instr_2`
    ...
    do `instr_ki`
```
Rules can have at least one instruction. Instructions are delimited with semicolons `;`. The last instruction in any rules must not have a dangling semicolon.

The first state to be declared is the initial state of the Turing machine.

Certain programs for Turing machine halt. In this language, halting states are declared with an empty state declaration similar to this:
```
HALT_STATE:
```
There can be multiple halting states.

Instructions for rules are already listed in the brief description of the Turing machine above. We list them here again:
- `shiftl` - Go to the cell to the left of the current cell.
- `shiftr` - Go to the cell to the right of the current cell.
- `noshift` - Do not shift cell.
- `write SYMBOL` - Write the symbol `SYMBOL` to the current cell.
- `erase` - Alias to the operation `write BLANK_SYM` where `BLANK_SYM` is the blank symbol.
- `goto STATE` - Go to the state `STATE`.

An example of a rule with the instructions above:
```
0 -> write 1; shiftl; goto B
```
 
The above rule states that if the symbol on the current cell is `0`, then write `1` on the current cell overwriting `0`, go to the cell left of the current cell and switch the state of the Turing machine to state `B`.

A parser was written for this programming language. It produces the action table or the transition function of the provided program.

##### Demo: Binary counter

In [2]:
from math import log2, log10

print("DEMO: Binary counter\n")

bin_count_prog = \
"""
symbol:
    █ (blank)
    0
    1

INCREMENT:
    █ -> write 0; noshift; goto INCREMENT
    0 -> write 1; noshift; goto INCREMENT
    1 -> write 0; shiftl;  goto CARRY

CARRY:
    █ -> write 1; noshift; goto RETURN
    0 -> write 1; noshift; goto RETURN
    1 -> write 0; shiftl;  goto CARRY

RETURN:
    █ -> shiftl; goto INCREMENT
    0 -> shiftr; goto RETURN
    1 -> shiftr; goto RETURN
"""

# Instatiate a Turing machine.
bin_count_tm = Turing(bin_count_prog)
# Load a blank tape.
bin_count_tm.load_tape()

count = 0
max_count = 20
lbound =  -int(log2(max_count))
fill_num = int(log10(max_count)) + 1

for _, state in bin_count_tm:
    if count == max_count:
        break

    # Print the tape only if the program is counting.
    if state == "INCREMENT":
        # Print the current number in the counter in decimal.
        print(f"{str(count).zfill(fill_num)}:  ", end="")
        bin_count_tm.print_tape(lbound=lbound, ubound=1, delim="")
        count += 1

DEMO: Binary counter

00:  ████0
01:  ████1
02:  ███10
03:  ███11
04:  ██100
05:  ██101
06:  ██110
07:  ██111
08:  █1000
09:  █1001
10:  █1010
11:  █1011
12:  █1100
13:  █1101
14:  █1110
15:  █1111
16:  10000
17:  10001
18:  10010
19:  10011


##### Demo: 3-state busy beaver

In [3]:
print("DEMO: 3-state Busy Beaver\n")

busy_beaver_3s_prog = \
"""
symbol:
    ·· (blank)   # In other implementations, this is "0".
    ██           # In other implementations, this is "1".

A:
    ·· -> write ██; shiftr; goto B
    ██ -> write ██; shiftr; goto HALT

B:
    ·· -> write ··; shiftr; goto C
    ██ -> write ██; shiftr; goto B

C:
    ·· -> write ██; shiftl; goto C
    ██ -> write ██; shiftl; goto A

HALT:
"""
# Instantiate a Turing machine.
bb_tm = Turing(busy_beaver_3s_prog)
# Load a blank tape.
bb_tm.load_tape()

# Run the Turing machine by iterating through it.
print("Step #")
for i, (_, _) in enumerate(bb_tm):
    # Print the left column.
    print(f"[{str(i+1).zfill(2)}]", end="   ")
    # Print the tape for the current iteration.
    bb_tm.print_tape(lbound=-2, ubound=6, delim="")

DEMO: 3-state Busy Beaver

Step #
[01]   ····██··········
[02]   ····██··········
[03]   ····██··██······
[04]   ····██████······
[05]   ····██████······
[06]   ··████████······
[07]   ··████████······
[08]   ··████████······
[09]   ··████████······
[10]   ··████████······
[11]   ··████████··██··
[12]   ··████████████··
[13]   ··████████████··
[14]   ··████████████··
